In [1]:
# PARAMETERS
ID = 'hercules_papers_l0_5000_mbert'  # used to determine output directory for checkpoints
DATASET = 'hercules.papers.l0.5000'  # all datasets are defined later in this script
EPOCHS = 5
BATCH_SIZE = 32
SEQ_LEN = 256
CHECKPOINT_FPATH = f"gs://hercules_zbeloki/checkpoints/{ID}/"

In [2]:
!pip install -q tensorflow-text
!pip install -q tf-models-official
!pip install -q tensorflow-addons

     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 1.8 MB 5.5 MB/s 
     |████████████████████████████████| 213 kB 54.9 MB/s 
     |████████████████████████████████| 47.6 MB 32 kB/s 
     |████████████████████████████████| 99 kB 8.2 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 90 kB 8.4 MB/s 
     |████████████████████████████████| 352 kB 32.9 MB/s 
     |████████████████████████████████| 1.2 MB 55.6 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 35.8 MB/s 


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from tensorflow_addons.metrics import MultiLabelConfusionMatrix
from tensorflow_addons.metrics import F1Score

from google.colab import auth, drive

import random
import os

print(tf.__version__)

2.6.0


In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("TPU")
else:
    strategy = tf.distribute.MirroredStrategy()
    print("No TPU")

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.90.13.122:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.90.13.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.90.13.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


TPU
REPLICAS:  8


In [5]:
# authenticate in Google in order to get access to Google Cloud Bucket
auth.authenticate_user()
!gcloud config set project hercules-315409

Updated property [core/project].


In [6]:
# The following datasets are read from Google Cloud Bucket from this account: zbeloki@gmail.com. Change it as you need.
DATASETS = {
    # format: (file_location, number_entries, delimiter)
    'sourceforge': {
        'train': ('gs://hercules_zbeloki/sourceforge.train_oversampled_max.csv', 55840, ','),
        'test': ('gs://hercules_zbeloki/sourceforge.test.csv', 8273, ','),
        'dev': ('gs://hercules_zbeloki/sourceforge.dev.csv', 8273, ','),
    },
    'bioprotocol': {
        'train': ('gs://hercules_zbeloki/bioprotocol.train_oversampled_max.csv', 4021, ','),
        'test': ('gs://hercules_zbeloki/bioprotocol.test.csv', 529, ','),
        'dev': ('gs://hercules_zbeloki/bioprotocol.dev.csv', 525, ','),
    },
    'hercules.papers.l0.5000': {
        'train': ('gs://hercules_zbeloki/papers_l0_5000.train.oversampled.tsv', 16851, '\t'),
        'test': ('gs://hercules_zbeloki/papers_l0_5000.test.tsv', 3611, '\t'),
        'dev': ('gs://hercules_zbeloki/papers_l0_5000.dev.tsv', 3611, '\t'),
    },
    'hercules.papers.l1.5000': {
        'train': ('gs://hercules_zbeloki/papers_l1_5000.train.oversampled.tsv', 112595, '\t'),
        'test': ('gs://hercules_zbeloki/papers_l1_5000.test.tsv', 18594, '\t'),
        'dev': ('gs://hercules_zbeloki/papers_l1_5000.dev.tsv', 18594, '\t'),
    },
    'hercules.papers.l2.5000': {
        'train': ('gs://hercules_zbeloki/papers_dataset_5000.train.oversampled.tsv', 843065, '\t'),
        'test': ('gs://hercules_zbeloki/papers_dataset_5000.test.tsv', 105054, '\t'),
        'dev': ('gs://hercules_zbeloki/papers_dataset_5000.dev.tsv', 105054, '\t'),
    },
    'hercules.papers.l2.10000': {
        'train': ('gs://hercules_zbeloki/papers_l2_10000.train.oversampled.tsv', 1164501, '\t'),
        'test': ('gs://hercules_zbeloki/papers_l2_10000.test.tsv', 141616, '\t'),
        'dev': ('gs://hercules_zbeloki/papers_l2_10000.dev.tsv', 141616, '\t'),
    },
    'hiekadi.location': {
        'train': ('gs://hercules_zbeloki/hiekadi.location.train.tsv', 151434, '\t'),
        'test': ('gs://hercules_zbeloki/hiekadi.location.test.tsv', 18929, '\t'),
        'dev': ('gs://hercules_zbeloki/hiekadi.location.dev.tsv', 18929, '\t'),
    },
}

fpath_train = DATASETS[DATASET]['train'][0]
fpath_test = DATASETS[DATASET]['test'][0]
fpath_dev = DATASETS[DATASET]['dev'][0]

n_train = DATASETS[DATASET]['train'][1]
n_test = DATASETS[DATASET]['test'][1]
n_dev = DATASETS[DATASET]['dev'][1]

delim_train = DATASETS[DATASET]['train'][2]
delim_test = DATASETS[DATASET]['test'][2]
delim_dev = DATASETS[DATASET]['dev'][2]

train_ds = tf.data.experimental.make_csv_dataset(fpath_train, BATCH_SIZE, num_epochs=1, field_delim=delim_train)
dev_ds = tf.data.experimental.make_csv_dataset(fpath_dev, BATCH_SIZE, num_epochs=1, shuffle=False, field_delim=delim_dev)

train_ds = train_ds.unbatch()
dev_ds = dev_ds.unbatch()

In [7]:
def prepare_multihot_label(feats):
  topic_labels = list(feats.values())[2:]
  multihot_label = tf.stack(topic_labels, axis=0)
  return (feats, multihot_label)

train_ds = train_ds.map(prepare_multihot_label)
dev_ds = dev_ds.map(prepare_multihot_label)

In [8]:
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)
dev_ds = dev_ds.batch(BATCH_SIZE, drop_remainder=True)

train_steps_per_epoch = int(n_train / BATCH_SIZE)
dev_steps_per_epoch = int(n_dev / BATCH_SIZE)

num_classes = next(iter(train_ds.take(1)))[1].shape[1]

In [9]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"
#bert_preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')  # BERT
bert_preprocess = hub.load('https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3')  # mBERT
with strategy.scope():
  #bert = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4', trainable=True)  # BERT
  bert = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4', trainable=True)  # mBERT

In [10]:
def make_bert_preprocess_model(sentence_features, seq_length=128):
    """Returns Model mapping string features to BERT inputs.
    Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

    Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
    """

    input_segments = [
        tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
        for ft in sentence_features]

    # Tokenize the text to word pieces.
    tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
    segments = [tokenizer(s) for s in input_segments]

    # Optional: Trim segments in a smart way to fit seq_length.
    # Simple cases (like this example) can skip this step and let
    # the next step apply a default truncation to approximately equal lengths.
    truncated_segments = segments

    # Pack inputs. The details (start/end token ids, dict of output tensors)
    # are model-dependent, so this gets loaded from the SavedModel.
    packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                            arguments=dict(seq_length=seq_length),
                            name='packer')
    model_inputs = packer(truncated_segments)
    
    return tf.keras.Model(input_segments,
                          {'input_word_ids': model_inputs['input_word_ids'],
                           'input_mask': model_inputs['input_mask'],
                           'all_segment_id': model_inputs['input_type_ids']})


preprocess_model = make_bert_preprocess_model(['text'], SEQ_LEN)

In [11]:
train_ds = train_ds.map(lambda feats, label: (preprocess_model(feats), label))
dev_ds = dev_ds.map(lambda feats, label: (preprocess_model(feats), label))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:585: UserWarning: Input dict contained keys ['id', 'Health Sciences', 'Life Sciences', 'Multidisciplinary', 'Physical Sciences', 'Social Sciences & Humanities'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


In [12]:
train_ds = train_ds.repeat(EPOCHS).cache().prefetch(tf.data.AUTOTUNE)

In [13]:
def build_classifier_model(max_seq_length=128):
    
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="all_segment_id")
    
    outputs = bert({'input_word_ids': input_word_ids, 'input_mask': input_mask, 'input_type_ids': segment_ids})
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(num_classes, activation='sigmoid', name='classifier')(net)
    
    return tf.keras.Model(inputs={'input_word_ids': input_word_ids, 'input_mask': input_mask, 'all_segment_id': segment_ids}, outputs=net)

In [14]:
def create_model():

    model = build_classifier_model(SEQ_LEN)
    
    loss = tf.keras.losses.BinaryCrossentropy()
    metrics = [F1Score(num_classes, average='micro', threshold=0.5, name='f1-micro'),
               F1Score(num_classes, average='macro', threshold=0.5, name='f1-macro'),
               tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall()]
    
    train_steps = train_steps_per_epoch * EPOCHS
    num_warmup_steps = int(0.1*train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics,
                  steps_per_execution=100)
    
    return model


with strategy.scope():
    model = create_model()
    
    model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=CHECKPOINT_FPATH,
        save_weights_only=True,
        monitor='val_f1-micro',
        mode='max',
        save_best_only=True)

In [17]:
history = model.fit(train_ds,
                    validation_data=dev_ds,
                    steps_per_epoch=train_steps_per_epoch,
                    validation_steps=dev_steps_per_epoch,
                    epochs=EPOCHS,
                    callbacks=[], #[model_checkpoint_cb],  # set this to save the checkpoint from the best epoch
                    verbose=1)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("AdamWeightDecay/gradients/PartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("clip_by_global_norm/clip_by_global_norm/_0:0", shape=(None, 768), dtype=float32), dense_shape=Tensor("AdamWeightDecay/gradients/PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("while/AdamWeightDecay/gradients/PartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("while/clip_by_global_norm/while/clip_by_global_norm/_0:0", shape=(None, 768), dtype=float32), dense_shape=Tensor("while/AdamWeightDecay/gradients/PartitionedCall:2", s

526/526 [==============================] - 121s 230ms/step - loss: 0.1564 - f1-micro: 0.8698 - f1-macro: 0.8645 - precision_1: 0.8730 - recall_1: 0.8666 - val_loss: 0.1436 - val_f1-micro: 0.9107 - val_f1-macro: 0.9159 - val_precision_1: 0.9215 - val_recall_1: 0.9002
Epoch 2/5
526/526 [==============================] - 39s 74ms/step - loss: 0.0575 - f1-micro: 0.9623 - f1-macro: 0.9648 - precision_1: 0.9695 - recall_1: 0.9553 - val_loss: 0.1540 - val_f1-micro: 0.9120 - val_f1-macro: 0.9170 - val_precision_1: 0.9153 - val_recall_1: 0.9087
Epoch 3/5
526/526 [==============================] - 39s 74ms/step - loss: 0.0365 - f1-micro: 0.9770 - f1-macro: 0.9786 - precision_1: 0.9816 - recall_1: 0.9725 - val_loss: 0.1561 - val_f1-micro: 0.9200 - val_f1-macro: 0.9252 - val_precision_1: 0.9137 - val_recall_1: 0.9264
Epoch 4/5
526/526 [==============================] - 39s 74ms/step - loss: 0.0206 - f1-micro: 0.9868 - f1-macro: 0.9875 - precision_1: 0.9892 - recall_1: 0.9844 - val_loss: 0.1699 - v

In [ ]:
# execute this only to load a pretrained checkpoint
with strategy.scope():
    model = create_model()
    model.load_weights(CHECKPOINT_FPATH)

In [18]:
test_ds_orig = tf.data.experimental.make_csv_dataset(fpath_test, BATCH_SIZE, num_epochs=1, shuffle=False, field_delim=delim_test)
topic_ids = list(test_ds_orig.element_spec.keys())[2:]

test_ds_orig = test_ds_orig.unbatch()
test_ds = test_ds_orig.map(prepare_multihot_label)
test_ds = test_ds.batch(1)
test_ds = test_ds.map(lambda feats, label: (preprocess_model(feats), label))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:585: UserWarning: Input dict contained keys ['id', 'Health Sciences', 'Life Sciences', 'Multidisciplinary', 'Physical Sciences', 'Social Sciences & Humanities'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


In [19]:
test_ds = test_ds.unbatch()
py_y_true = []
for feats, label in test_ds:
  py_y_true.append(label)
y_true = tf.stack(py_y_true, axis=0)

In [20]:
BATCH_SIZE = 16
test_ds = test_ds.batch(BATCH_SIZE, drop_remainder=True)
test_steps_per_epoch = int(n_test / BATCH_SIZE)
y_pred = model.predict(test_ds, verbose=1, steps=test_steps_per_epoch)

225/225 [==============================] - 20s 87ms/step


In [21]:
y_true = tf.round(y_true)
y_pred = tf.cast(tf.round(y_pred), dtype=tf.int32)

len_drop_remainder = int(len(y_true) / BATCH_SIZE) * BATCH_SIZE
y_true = y_true[:len_drop_remainder]

In [22]:
metric = MultiLabelConfusionMatrix(num_classes=num_classes)
metric.update_state(y_true, y_pred)
result = metric.result()

In [23]:
print(f"TOPIC\tP (TP)\tR (FP)\tF1 (FN)")
total_tn, total_fp, total_fn, total_tp = 0, 0, 0, 0
all_p, all_r, all_f1 = [], [], []
for i_class, conf_matrix in enumerate(result):
  tn, fp = conf_matrix[0]
  fn, tp = conf_matrix[1]
  #print(f"TP:{tp}, FP:{fp}, FN:{fn}")
  p = tp / (tp + fp) if tp > 0 else 0.0
  r = tp / (tp + fn) if tp > 0 else 0.0
  f1 = (2 * p * r) / (p + r) if p + r > 0 else 0.0
  all_p.append(p)
  all_r.append(r)
  all_f1.append(f1)
  print(f"{topic_ids[i_class]}\t{p:.3f}\t{r:.3f}\t{f1:.3f}")
  #print(f"{'':<16}\t{tp}\t{fp}\t{fn}")
  total_tn += tn
  total_fp += fp
  total_fn += fn
  total_tp += tp
p_micro = total_tp / (total_tp + total_fp)
r_micro = total_tp / (total_tp + total_fn)
f1_micro = (2 * p_micro * r_micro) / (p_micro + r_micro)
p_macro = sum(all_p) / len(all_p)
r_macro = sum(all_r) / len(all_r)
f1_macro = sum(all_f1) / len(all_f1)
print(f"TOTAL (micro)\t{p_micro:.3f}\t{r_micro:.3f}\t{f1_micro:.3f}")
print(f"TOTAL (macro)\t{p_macro:.3f}\t{r_macro:.3f}\t{f1_macro:.3f}")

TOPIC	P (TP)	R (FP)	F1 (FN)
Health Sciences	0.899	0.904	0.901
Life Sciences	0.868	0.907	0.887
Multidisciplinary	0.989	0.983	0.986
Physical Sciences	0.914	0.922	0.918
Social Sciences & Humanities	0.921	0.917	0.919
TOTAL (micro)	0.913	0.922	0.917
TOTAL (macro)	0.918	0.926	0.922


# Error analysis

In [24]:
pids = []
for p in test_ds_orig:
  pids.append(p['id'].numpy().decode('utf-8'))

In [25]:
id_preds = list(zip(pids, y_pred.numpy().tolist(), y_true.numpy().tolist()))
random.shuffle(id_preds)

In [26]:
N_PRINT = 15
print(', '.join([ tid for tid in topic_ids]))
print()
print(f"{'':<24}{' '.join([ tid[:6].upper() for tid in topic_ids])}")
print()
for pid, _pred, _true in id_preds[:N_PRINT]:
  print(f"{pid:<24}{' '.join([ str(p)+'     ' for p in _pred ])}")
  print(f"{'':<24}{' '.join([ str(p)+'     ' for p in _true ])}")
  print()

Health Sciences, Life Sciences, Multidisciplinary, Physical Sciences, Social Sciences & Humanities

                        HEALTH LIFE S MULTID PHYSIC SOCIAL

2-s2.0-77957805316      1      1      0      0      0     
                        1      1      0      0      0     

2-s2.0-84971222576      0      0      1      0      0     
                        0      0      1      0      0     

2-s2.0-34547412109      0      0      1      0      0     
                        0      0      1      0      0     

2-s2.0-33748771222      0      0      0      1      0     
                        0      1      0      1      0     

2-s2.0-72449172130      0      0      0      1      1     
                        0      0      0      1      1     

2-s2.0-84876412208      1      0      0      0      0     
                        1      0      0      0      0     

2-s2.0-84981312909      0      0      1      1      0     
                        0      0      1      1      0     

2-s2.0-